# Movie genres 
## Baseline 1: Fasttext & LightGBM

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

from lightgbm import LGBMClassifier
import fasttext.util

ModuleNotFoundError: No module named 'fasttext'

## Загружаем данные

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission_most_popular.csv')

## Работаем с обучающей выборкой

### Fasttext embeddings

In [3]:
# Загружаем предобученную модель из библиотеки fasttext

fasttext.util.download_model('en', if_exists='ignore')
ft = fasttext.load_model('cc.en.300.bin')

ModuleNotFoundError: No module named 'fasttext'

In [4]:
# Чистим диалоги - оставляем только буквы и пробелы

train['dialogue'] = train.dialogue.apply(lambda s: ''.join(x for x in s if (x.isalpha() | x.isspace())))
test['dialogue'] = test.dialogue.apply(lambda s: ''.join(x for x in s if (x.isalpha() | x.isspace())))

# С помощью предобученной модели получаем для каждого диалога векторное представление 

X = train.dialogue.apply(lambda x: ft.get_sentence_vector(x))

# переводим массив списков в numpy array  

X = np.array(X.tolist())

## Преобразовываем вектор ответов к виуду multilabel

In [7]:
mlb = MultiLabelBinarizer()

# вектор ответов у нас в формате строка. Чтобы "выполнить" строку и получить список, нам понадобится функция eval

train.genres = train.genres.apply(eval)

# создаем объект pandas DataFrame: данные - результат преобразования MultiLabelBinarizer, названия столбцов - жанры

y = pd.DataFrame(mlb.fit_transform(train.genres), columns=mlb.classes_)

y.head()

,action,adventure,animation,biography,comedy,crime,drama,family,fantasy,history,horror,music,musical,mystery,romance,sci-fi,sport,thriller,war,western
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


## Оценим качество

In [8]:
# Разбиваем выборку для валидации (holdout)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Здесь мы будем использовать sklearn.multiclass.OneVsRestClassifier, который позволяет реализовывать стратегию one-vs-all - обучать отдельную модель для каждого класса, где в качестве положительных примеров берутся все объекты этого класса, в качестве отрицательных - все остальные объекты. 

In [9]:
# Оценка качества

ovrc = OneVsRestClassifier(LGBMClassifier(), n_jobs=-1)
ovrc.fit(X_train, y_train)
preds = ovrc.predict(X_test)

f1_score(y_test, preds, average='samples')

0.41707350371341195

## Готовим сабмит

In [10]:
# Обучающий датасет мы уже обработали

X_train = X.copy()
y_train = y.copy()

# Для диалогов тестовой выборки получаем векторное представление и формируем датасет

X_test = test.dialogue.apply(lambda x: ft.get_sentence_vector(x))
X_test = np.array(X_test.tolist())

# обучаем модель и получаем предсказания для тестовой выборки

ovrc.fit(X_train, y_train)
preds = ovrc.predict(X_test)

# формируем файл с предсказаниями

sample_submission['genres'] = mlb.inverse_transform(preds)
sample_submission['genres'] = sample_submission['genres'].apply(lambda x: ' '.join((x)))
sample_submission.to_csv('benchmark_1.csv', index=False)

Public 0.40029 

Private 0.40635